### Update some records of Feature Service

The country geometries simplified for gadm level 0 is failing to load on the FE.  
This geometries have been simplifies previously using mapshaper or ArcGIS Pro.  
There is a previous layer used in the NRC that does not fail, we could use that layer for gadm0.
The countries failing so far are Russia, USA and Canada (classic, the big countries)

current gadm0 layer: https://eowilson.maps.arcgis.com/home/item.html?id=e9b2595e8ee2406b981fd7c00f77df9c  
NRC layer: https://eowilson.maps.arcgis.com/home/item.html?id=3c7c5f75cc184a4ca89fe9c8c2154da5

In [11]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

In [12]:
env_path = "../../.env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [13]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [14]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

In [22]:
layers= {'gadm0':'e9b2595e8ee2406b981fd7c00f77df9c',
        'nrc':'3c7c5f75cc184a4ca89fe9c8c2154da5',
        'gadm1':'3f89b653de41428a8166d51f383f0950'}

In [15]:
def getFlayerFromID(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    sdf = pd.DataFrame.spatial.from_layer(flayer)
    return sdf

In [16]:
pre= getFlayerFromID(layers['gadm0'])

In [17]:
nrc= getFlayerFromID(layers['nrc'])

In [32]:
print(len(nrc))
nrc.columns

252


Index(['OBJECTID', 'GID_0', 'NAME_0', 'Shape__Area', 'Shape__Length', 'SHAPE'], dtype='object')

In [33]:
print(len(pre))
pre.columns

255


Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'reptiles', 'amphibians',
       'mammals', 'birds', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'country_size', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban', 'ObjectId',
       'Shape__Area', 'Shape__Length', 'wdpa_mol_id', 'SHAPE'],
      dtype='object')

In [49]:
selp = pre[pre['GID_0'].isin(['RUS','CAN','USA'])]
seln = nrc[nrc['GID_0'].isin(['RUS','CAN','USA'])]

In [ ]:
display(seln.spatial)

In [ ]:
display(selp.spatial)

In [70]:
seln

,OBJECTID,GID_0,NAME_0,Shape__Area,Shape__Length,SHAPE
39,40,CAN,Canada,1694.025240,2210.684826,"{""rings"": [[[-87.035281988421, 65.479153994765..."
248,253,USA,United States,1116.381362,719.210324,"{""rings"": [[[-160.021144980627, -0.39805494820..."
249,400,RUS,Russia,2908.001129,1478.397360,"{""rings"": [[[131.87329129417, 42.9569360103709..."


In [99]:
### Layer to update (gadm0 layer)
item_id = 'e9b2595e8ee2406b981fd7c00f77df9c' ### gadm0 layer
item = gis.content.get(item_id)
flayer = item.layers[0]
fset = flayer.query(where ="GID_0 IN ('CAN','RUS','USA')")

### Layer to retrieve geometries from
new_fset = gis.content.get('3c7c5f75cc184a4ca89fe9c8c2154da5').layers[0].query(where ="GID_0 IN ('CAN','RUS','USA')")
new_fset_features = gis.content.get('3c7c5f75cc184a4ca89fe9c8c2154da5').layers[0].query(where ="GID_0 IN ('CAN','RUS','USA')").features
new_flayer_rows = new_fset.sdf

In [100]:
new_fset.fields

[{'name': 'OBJECTID',
  'type': 'esriFieldTypeOID',
  'alias': 'OBJECTID',
  'sqlType': 'sqlTypeOther',
  'domain': None,
  'defaultValue': None},
 {'name': 'GID_0',
  'type': 'esriFieldTypeString',
  'alias': 'GID_0',
  'sqlType': 'sqlTypeOther',
  'length': 8000,
  'domain': None,
  'defaultValue': None},
 {'name': 'NAME_0',
  'type': 'esriFieldTypeString',
  'alias': 'NAME_0',
  'sqlType': 'sqlTypeOther',
  'length': 8000,
  'domain': None,
  'defaultValue': None},
 {'name': 'Shape__Area',
  'type': 'esriFieldTypeDouble',
  'alias': 'Shape__Area',
  'sqlType': 'sqlTypeDouble',
  'domain': None,
  'defaultValue': None},
 {'name': 'Shape__Length',
  'type': 'esriFieldTypeDouble',
  'alias': 'Shape__Length',
  'sqlType': 'sqlTypeDouble',
  'domain': None,
  'defaultValue': None}]

In [133]:
### https://community.esri.com/t5/arcgis-online-questions/updating-polygon-features-from-a-feature-layer-in/td-p/1043566
from arcgis import geometry
features_to_be_added = []
template_hostedFeature = deepcopy(fset.features[0])

for index, row in new_flayer_rows.iterrows():
    feature_to_be_updated = deepcopy(template_hostedFeature)
    input_geometry = new_fset_features[index].geometry
    
    output_geometry = geometry.project(geometries = [input_geometry],
                                       in_sr = new_fset.spatial_reference['wkid'], 
                                       out_sr = fset.spatial_reference['wkid'], 
                                       gis = gis)
    # assign the updated values
    feature_to_be_updated.geometry = output_geometry
    #feature_to_be_updated.attributes['OBJECTID'] = int(row['OBJECTID'])
    
    # #add this to the list of features to be updated
    features_to_be_added.append(feature_to_be_updated)


In [134]:
#fset.features[0]
fset.spatial_reference

{'wkid': 102100, 'latestWkid': 3857}

In [135]:
#features_to_be_added[0].geometry
new_fset.spatial_reference

{'wkid': 4326, 'latestWkid': 4326}

In [145]:
flayer.edit_features(adds = features_to_be_added) 
#### In order to add the geometries instead of adding them I should use: 
###flayer.edit_features(updates = features_to_be_added) but check that the geometries are correct before adding anything

{'addResults': [{'objectId': 256,
   'uniqueId': 256,
   'globalId': None,
   'success': True},
  {'objectId': 257, 'uniqueId': 257, 'globalId': None, 'success': True},
  {'objectId': 258, 'uniqueId': 258, 'globalId': None, 'success': True}],
 'updateResults': [],
 'deleteResults': []}

In [ ]:
# This added new rows with a copy of the data and new geometries, but it did not update teh geoemtry of the rows.

In [163]:
item_id = 'e9b2595e8ee2406b981fd7c00f77df9c' ### gadm0 layer
item = gis.content.get(item_id)
flayer = item.layers[0]
fset = flayer.query(where ="GID_0 IN ('CAN','RUS','USA')")
df = fset.sdf

In [154]:
df.columns

Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'reptiles', 'amphibians',
       'mammals', 'birds', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'country_size', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban', 'ObjectId',
       'Shape__Area', 'Shape__Length', 'wdpa_mol_id', 'SHAPE'],
      dtype='object')

In [155]:
df[['GID_0','SHAPE','ObjectId']]

,GID_0,SHAPE,ObjectId
0,CAN,"{""rings"": [[[-7765692.92845951, 17917763.32315...",40
1,RUS,"{""rings"": [[[6450032, 16819540], [6449632, 168...",190
2,USA,"{""rings"": [[[-17646464, 12004764], [-17646864,...",235
3,CAN,None,256
4,CAN,None,257
5,CAN,None,258


In [162]:
flayer.delete_features(where="objectid > 255")

{'deleteResults': [{'objectId': 256,
   'uniqueId': 256,
   'globalId': None,
   'success': True},
  {'objectId': 257, 'uniqueId': 257, 'globalId': None, 'success': True},
  {'objectId': 258, 'uniqueId': 258, 'globalId': None, 'success': True}]}

In [164]:
df

,GID_0,NAME_0,MOL_ID,AREA_KM2,reptiles,amphibians,mammals,birds,percentage_protected,population_sum,...,country_size,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,ObjectId,Shape__Area,Shape__Length,wdpa_mol_id,SHAPE
0,CAN,Canada,40,9955221,"[{""SliceNumber"":145.0,""percentage_presence"":0....","[{""SliceNumber"":3.0,""percentage_presence"":0.0}...","[{""SliceNumber"":129.0,""percentage_presence"":53...","[{""SliceNumber"":36.0,""percentage_presence"":7.4...",13.10794,36769980,...,1,0,6.03,0.13,0.27,40,5.106786e+13,6.416987e+08,"[207024, 191268, 191232, 191143, 191102, 21414...","{""rings"": [[[-7765692.92845951, 17917763.32315..."
1,RUS,Russia,190,16925820,"[{""SliceNumber"":1.0,""percentage_presence"":0.0}...","[{""SliceNumber"":815.0,""percentage_presence"":7....","[{""SliceNumber"":129.0,""percentage_presence"":16...","[{""SliceNumber"":29.0,""percentage_presence"":2.7...",11.29689,140597000,...,1,0.12,11.10,3.32,0.24,190,8.275980e+13,3.564010e+08,"[216284, 41227, 32190, 61584, 164288, 127907, ...","{""rings"": [[[6450032, 16819540], [6449632, 168..."
2,USA,United States,235,9472976,"[{""SliceNumber"":145.0,""percentage_presence"":4....","[{""SliceNumber"":3.0,""percentage_presence"":18.0...","[{""SliceNumber"":129.0,""percentage_presence"":18...","[{""SliceNumber"":36.0,""percentage_presence"":66....",16.06911,342629300,...,1,0.85,26.35,20.6,4.37,235,2.172280e+13,2.024955e+08,"[170882, 111561, 131362, 151167, 206948, 13146...","{""rings"": [[[-17646464, 12004764], [-17646864,..."


In [ ]:
### Try different approach to update (not append values)

In [168]:
features_for_update = [] #list containing corrected features

all_features = fset.features

In [169]:
for country_id in df['GID_0']:
    print(country_id)

CAN
RUS
USA


In [175]:
for country_id in df['GID_0']:
    # get the feature to be updated
    original_feature = [f for f in all_features if f.attributes['GID_0'] == country_id][0]
    feature_to_be_updated = deepcopy(original_feature)
    
    
    # get the matching row from dev
    matching_row = new_flayer_rows.where(df.GID_0 == country_id).dropna()
    
    feature_to_be_updated.attributes['OBJECTID'] = int(matching_row['OBJECTID'])
    
    input_geometry = matching_row.SHAPE
    
    output_geometry = geometry.project(geometries = [input_geometry],
                                       in_sr = new_fset.spatial_reference['wkid'], 
                                       out_sr = fset.spatial_reference['wkid'], 
                                       gis = gis)
    # assign the updated values
    feature_to_be_updated.geometry = output_geometry
    
    #add this to the list of features to be updated
    features_for_update.append(feature_to_be_updated)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [173]:
matching_row.SHAPE

0    {"rings": [[[-87.035281988421, 65.479153994765...
Name: SHAPE, dtype: geometry

In [ ]:
flayer_prod.edit_features(updates= features_for_update)